# Teste Renner

## Importação da bibliotecas

In [1]:
import pandas as pd

## Leitura do arquivo

In [2]:
caminho = '/home/diegopaes/ml_projects/teste_renner/teste_renner/Teste Pesquisa Operacional - Dados.xlsx'
df = pd.read_excel(caminho)

## Definição de funções

In [3]:
def organizar_caixas(caixas, capacidade_maxima):
    ondas = []
    item_por_onda = {}

    # Função principal que organiza as caixas em ondas
    for c, ic in caixas.items():
        melhor_onda = melhor_onda_encontrada(caixas, ondas, ic, capacidade_maxima)
        if melhor_onda is not None:
            caixa_alocanda_na_onda(c, ic, melhor_onda, caixas, item_por_onda, ondas)
        else:
            criacao_nova_onda(c, ic, caixas, item_por_onda, ondas)

    return ondas, item_por_onda

# encontra a melhor onda com a menor quantidade de novos itens
def melhor_onda_encontrada(caixas, ondas, itens_na_caixa, capacidade_maxima):
    melhor_onda = None
    menor_novos_itens = float('inf')
    tot_itens_caixa = total_itens_caixa(itens_na_caixa)
    
    for onda in ondas:
        tot_itens_onda = total_itens_onda(onda, caixas)
        novo_total_itens_caixa = tot_itens_onda + tot_itens_caixa
        if novo_total_itens_caixa <= capacidade_maxima:
            novos_itens = calcular_novos_itens(onda, itens_na_caixa, caixas)
            if novos_itens < menor_novos_itens:
                melhor_onda = onda
                menor_novos_itens = novos_itens

    return melhor_onda

# calcula o total de itens em uma caixa
def total_itens_caixa(itens_na_caixa):
    tot_quantidade = 0
    for _, quantidade in itens_na_caixa:
        tot_quantidade += quantidade
    return tot_quantidade

# calcula o total de itens em uma onda
def total_itens_onda(onda, caixas):
    total = 0
    for c in onda:
        for item in caixas[c]:
            total += item[1]
    return total

# calcula quantos itens novos seriam inseridos em uma onda
def calcular_novos_itens(onda, itens_na_caixa, caixas):
    itens_existentes = item_existente(onda, caixas)
    novos_itens = novo_item(itens_na_caixa, itens_existentes)
    return len(novos_itens)
    
def item_existente(onda, caixas):
    item_unico = set()
    for caixa in onda:
        for item, _ in caixas[caixa]:
            item_unico.add(item)
    return item_unico
    
def novo_item(itens_na_caixa, itens_existentes):
    itens_novos = []
    for item, _ in itens_na_caixa:
        if item not in itens_existentes:
            itens_novos.append(item)
    return itens_novos
    
# aloca uma caixa em uma onda existente
def caixa_alocanda_na_onda(caixa, itens_na_caixa, melhor_onda, caixas, item_por_onda, ondas):
    melhor_onda.append(caixa)
    atualizar_itens_por_onda(itens_na_caixa, item_por_onda, ondas.index(melhor_onda))

# Função para criar uma nova onda e alocar a caixa nela
def criacao_nova_onda(caixa, itens_na_caixa, caixas, item_por_onda, ondas):
    nova_onda = [caixa]
    ondas.append(nova_onda)
    atualizar_itens_por_onda(itens_na_caixa, item_por_onda, len(ondas) - 1)

# Função para atualizar o mapeamento de itens por onda
def atualizar_itens_por_onda(itens_na_caixa, item_por_onda, onda_index):
    for item, _ in itens_na_caixa:
        if item not in item_por_onda:
            item_por_onda[item] = set()
        item_por_onda[item].add(onda_index)
        
def criacao_dataset(ondas):
    dataset = []
    for i, onda in enumerate(ondas):
        for caixa in onda:
            dataset.append((caixa, i+1))
    return pd.DataFrame(dataset, columns=['Caixa', 'Onda'])


## Modelo

In [4]:
capacidade_maxima = 2000

# Cria uma lista de caixas, onde cada caixa contém um conjunto de itens e suas quantidades
caixas = df.groupby('Caixa Id').apply(lambda x: list(zip(x['Item'], x['Peças']))).to_dict()

# Chama a função de organização de caixas
ondas, item_por_onda = organizar_caixas(caixas, capacidade_maxima)

# Cria um DataFrame para exportar as ondas
ondas_df = criacao_dataset(ondas)

# Exportar para um novo arquivo Excel
nome_arquivo = 'caixas_em_ondas_minimizando_itens.xlsx'
ondas_df.to_excel(nome_arquivo, index=False)

/tmp/ipykernel_393811/722809153.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  caixas = df.groupby('Caixa Id').apply(lambda x: list(zip(x['Item'], x['Peças']))).to_dict()
